In [1]:
import csv
import json
import base64

from urllib.request import urlopen

key = "55228d9d4f216ad5972abd1be3c4c2e3"

In [2]:
def get_bill_ids(filePath):
    """Get bill ids from a csv file"""

    bill_ids = []
    with(open(filePath, 'r')) as f:
        reader = csv.reader(f)
        for row in reader:
            bill_ids.append(row[0])
    return bill_ids        

In [3]:
VA_bill_ids = get_bill_ids("VA/2024-2024_1st_Special_Session/csv/bills.csv")
PA_bill_ids = get_bill_ids("PA/2023-2024_Regular_Session/csv/bills.csv")

In [4]:
VA_bill_ids[:10]

['bill_id',
 '1877734',
 '1877735',
 '1882815',
 '1883989',
 '1877732',
 '1877737',
 '1877730',
 '1877736',
 '1877727']

In [5]:
def get_text_from_bill_id(bill_id):
    """API pull to get text from bill id
    returns None if bill id is not found
    returns empy string if no text is unavailable
    returns encoded text (with base64) if available
    """
    
    urlGetBill = f"https://api.legiscan.com/?key={key}&op=getBill&id={bill_id}"
    r1 = urlopen(urlGetBill).read().decode()
    data1 = json.loads(r1)
    if data1["status"] == "ERROR":
        return None
    if data1["bill"]["texts"] == []:
        return ""
    else:
        doc_id = data1["bill"]["texts"][0]["doc_id"]
        urlGetText = f"https://api.legiscan.com/?key={key}&op=getBillText&id={doc_id}"
        r2 = urlopen(urlGetText).read().decode()
        data2 = json.loads(r2)
        textEncoded = data2["text"]["doc"]
        return textEncoded

In [6]:
PA_bill_ids[4]

'1745716'

In [7]:
data = get_text_from_bill_id(PA_bill_ids[4])
base64.b64decode(data)
# with open("sampleText.txt", "w") as f:
#     f.write(base64.b64decode(data).decode())


b'%PDF-1.4\n%\xc3\xa4\xc3\xbc\xc3\xb6\xc3\x9f\n2 0 obj\n<</Length 3 0 R/Filter/FlateDecode>>\nstream\nx\x9c\x95X\xc9\x8e\xe3F\x0c\xbd\xfb+t\x1e\xa0\x9d"Y\x8b\n0\x0c\xc8[\x80\xdc&h \x87 \xb7\xcc$\x082\x012\x97\xfc~\x1eYeY\x96T\xf2L7\xdaR-,\x92\xefq\xa9\xb6\xdbS\xf7\xdf\xee\xdf\xce\xe17\x12w)s\xf7\xf5\xd3\xee\x97\x0f\xdd?\xbb\x8f\x9d\xce\'\xf1\xfbhk\x9c\xc7%g\x02?\xffX_\xbe\xfe\xb1\x0bq\xef\xbb\xe4\x1d\xf6~\xd9I\x9f\xf6\\G\x7fwu$\xd9Fec\x1d\xfc\xb9;}\xa8\xa2\x92`\xca(j\xa3\x87\xa8\x9f\x8a\xfa\x87\xe8T\xd1\x97]Hi\x9fF\xb5uT5=\x1b1\x15\xae\x8a\xef\xdb\x8b\xe2\xfb\xc8?\tO4\x17p\x96\xa0\x95Y\x83\xe4\xf4\xbeK\x05\x15\xde\xe7\xee\xfd\xf7\xee\x87\x1bu\xd8\xfe\xfe\xf9\xe0\xe8\xf8\xfe\xd7\xee\xfa\xbe\xfb8\x13\xf0\x14\xa0#\xf9\x08\xed\xcf\x12\xec\xc4y\x17\\t\to\xbd\xcb\x8e0\x1a\xdc\xa9qRp\xd9\xa0Y\x9eDM\xed\x81\xe5Y;w\xd4W\x19\xe8oI\x05i\xd8\xdc\xd2C1\xed\xfb.\xb9\xfe\x8e\x8b\x14\x89_U\xe4\x8d<\xbc\xc5#\x1f\xa0\xf2\x8d\xe2\xc1\xf9\xe3\x1b\xf3\xc1\x85\xb2V\'\xe3\xb8\xa5?\xb8t|\xf3X\xea\x8f\xba/\x97

In [8]:
#Create a CSV file with the bill_id and the encoded text
bill_texts = []
bill_texts.append(["state","bill_id", "text"])
count = 0
for bill_id in VA_bill_ids: #skip the first line since its the header
    if bill_id == "bill_id":
        continue
    billText = get_text_from_bill_id(bill_id)
    count += 1
    if count % 100 == 0:
        print(count)
    bill_texts.append(["VA", bill_id, billText]) #If empty string then text was unavailable on LegiScan

# for bill_id in PA_bill_ids: #skip the first line since its the header
#     if bill_id == "bill_id":
#         continue
#     billText = get_text_from_bill_id(bill_id)
#     bill_texts.append(["PA", bill_id, billText]) #If empty string then text was unavailable on LegiScan
    
with open("bill_texts.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(bill_texts)
        


100
200
